In [1]:
import datetime
import pandas
import urllib
import json
import time
import sys
import datetime
import time
import requests
from yahoo_earnings_calendar import YahooEarningsCalendar
import numpy as np
import os

In [2]:
today = datetime.date.today()

path = os.getcwd()

today = datetime.date.today()

path = os.getcwd()

print(path)

#get data from bloomberg
def get_data(url):
	req = urllib.request.Request(url)
	response = urllib.request.urlopen(req)
	file = response.read()
	result = json.loads(file)
	result = pandas.DataFrame([result])
	return result

sandbox = "https://sandbox.iexapis.com"

sb_api_key = "Tpk_db75edaa9a9b4b2891dff83893eff6d6"

production = "https://cloud.iexapis.com/"

prod_api_key = "pk_1de77928d18b46e1b733bfa018321964"

base = datetime.datetime.today()

date_calendar = [base + datetime.timedelta(days=x) for x in range(7)]

date_from = date_calendar[0].date()

date_to = date_calendar[-1].date()

yec = YahooEarningsCalendar()

earnings = yec.earnings_between(date_from, date_to)

# earnings = json.loads(earnings)

eps_df = pandas.DataFrame.from_records(earnings)

/home/wc/Azimuth/Trader/Notebooks


In [3]:
ticker_ary = []
epsEstimate = []
marketCap = []
peRatio = []
price = []
debt = []
priceToSales = []
priceToBook = []
sharesOutstanding = []
day5ChangePercent = []

In [4]:
eps_df = eps_df[['companyshortname', 'ticker', 'epsestimate', 'startdatetime', 'startdatetimetype']]

In [5]:
print(len(eps_df))

800


In [8]:
eps_df.head(20)

,companyshortname,ticker,epsestimate,startdatetime,startdatetimetype
0,Berkshire Hathaway Inc,BRKB,NaN,2020-02-22T18:30:00.000Z,BMO
1,180 Degree Capital Corp,TURN,NaN,2020-02-25T02:00:00.000Z,AMC
2,Rent-A-Center Inc,RCII,0.57,2020-02-25T02:00:00.000Z,AMC
3,Altisource Portfolio Solutions SA,ASPS,0.47,2020-02-24T15:59:00.000Z,TNS
4,Armstrong World Industries Inc,AWI,0.86,2020-02-24T18:30:00.000Z,BMO
5,Argo Group International Holdings Ltd,ARGO,-1.63,2020-02-25T02:00:00.000Z,AMC
6,American National Insurance Co,ANAT,NaN,2020-02-24T15:59:00.000Z,TNS
7,Arlo Technologies Inc,ARLO,-0.31,2020-02-24T15:59:00.000Z,TNS
8,Preferred Apartment Communities Inc,APTS,-0.69,2020-02-25T02:00:00.000Z,AMC
9,Adams Diversified Equity Fund Inc,ADX,NaN,2020-02-25T02:00:00.000Z,AMC


In [12]:

for i in range(len(eps_df)):
	ticker = eps_df['ticker'].loc[i]
	price_url = "https://sandbox.iexapis.com/v1/stock/{}/quote/?token=Tpk_db75edaa9a9b4b2891dff83893eff6d6".format(str(eps_df['ticker'].loc[i]).lower())
	stats_url = "https://sandbox.iexapis.com/v1/stock/{}/advanced-stats/?token=Tpk_db75edaa9a9b4b2891dff83893eff6d6".format(str(eps_df['ticker'].loc[i]).lower())

	try:
		stats_result = get_data(stats_url)
		price_result = get_data(price_url)
		priceToSales.append(stats_result["priceToSales"].values[0])
		priceToBook.append(stats_result["priceToBook"].values[0])
		sharesOutstanding.append(stats_result["sharesOutstanding"].values[0])
		day5ChangePercent.append(stats_result["day5ChangePercent"].values[0])
		debt.append(stats_result["currentDebt"].values[0])
		epsEstimate.append(eps_df['epsestimate'].loc[i])
		marketCap.append(price_result["marketCap"].values[0])
		peRatio.append(price_result["peRatio"].values[0])
		price.append(price_result["latestPrice"].values[0])
		ticker_ary.append(ticker)
	except Exception as e:
		print(e)
		pass



HTTP Error 404: Not Found
HTTP Error 404: Not Found


In [13]:

print(len(ticker_ary), len(marketCap), len(peRatio), len(price), len(debt), len(priceToSales), len(priceToBook), len(sharesOutstanding), len(day5ChangePercent))

798 798 798 798 798 798 798 798 798


In [14]:
fin_analysis = pandas.DataFrame({"ticker": ticker_ary, "epsEstimate": epsEstimate, "marketCap": marketCap, 
                                 "peRatio": peRatio, "price": price, "debt": debt, "priceToSales": priceToSales , 
                                 "priceToBook" : priceToBook, "sharesOutstanding" : sharesOutstanding, "day5ChangePercent" : day5ChangePercent})
fin_analysis["epsEstimate"] = fin_analysis["epsEstimate"]
fin_analysis["price"] = fin_analysis["price"]
fin_analysis["peRatio"] = fin_analysis["peRatio"]
fin_analysis["priceToBook"] = fin_analysis["priceToBook"]
fin_analysis["priceToSales"] = fin_analysis["priceToSales"]
fin_analysis["debt"] = fin_analysis["debt"]
fin_analysis["day5ChangePercent"] = fin_analysis["day5ChangePercent"]




In [15]:
fin = fin_analysis[fin_analysis["epsEstimate"] > 0 ]
fin = fin_analysis[(fin_analysis["price"] < 50) & (fin_analysis["price"] > 0)]
fin = fin_analysis[(fin_analysis["peRatio"] < 11) & (fin_analysis["peRatio"] != -1)]
fin = fin_analysis[fin_analysis["priceToBook"] < 10]
fin = fin_analysis[fin_analysis["priceToSales"] < 10 & (fin_analysis["priceToSales"] > 0)]
fin = fin_analysis[fin_analysis["debt"] > 0]
fin = fin_analysis[fin_analysis["day5ChangePercent"] > 0]


In [16]:
fin

,ticker,epsEstimate,marketCap,peRatio,price,debt,priceToSales,priceToBook,sharesOutstanding,day5ChangePercent
0,TURN,-1.00,67584268,-24.32,2.20,-1.0,-9.80,0.842820,31744997.0,0.002848
3,AWI,0.86,5227023246,23.94,111.74,56857540.0,5.51,20.090388,48595270.0,0.026042
5,ANAT,-1.00,3152219193,9.43,115.16,-1.0,-1.00,0.588438,27065291.0,0.033173
7,APTS,-0.69,586541412,-6.41,12.56,188800365.0,1.44,0.358719,47113008.0,0.022720
10,AMRN,-0.02,6503801465,-69.63,18.59,35425071.0,28.04,42.460630,365050394.0,0.018459
...,...,...,...,...,...,...,...,...,...,...
791,MTBCP,-1.00,356463909,-1.00,28.60,-1.0,-1.00,-1.000000,12390529.0,0.021894
794,MRSN,-0.38,440222132,-9.68,9.11,-1.0,-1.00,48.133365,50039547.0,0.132506
795,SMLP,0.06,220597995,-6.21,2.68,-1.0,0.44,0.178359,83350543.0,0.078027
796,TPRE,0.39,1046824722,-8.09,11.12,-1.0,-1.00,0.871403,96545004.0,0.005520


In [22]:
import boto3
from io import StringIO

In [23]:
s3 = boto3.client("s3")

In [25]:
bucket = 'azimuthcollector'
csv_buffer = StringIO()
fin_analysis.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, f'azimuth_trades_{today}.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'B47BD14A930636CA',
  'HostId': 'SCm7shLhIru/nj8n7cIKMRVmWGfQyWMz39Jk5ZTGpIOsQFgq5YZy9MXB6CMVSMv22WZ+eQiIpc4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'SCm7shLhIru/nj8n7cIKMRVmWGfQyWMz39Jk5ZTGpIOsQFgq5YZy9MXB6CMVSMv22WZ+eQiIpc4=',
   'x-amz-request-id': 'B47BD14A930636CA',
   'date': 'Mon, 20 Jan 2020 18:56:09 GMT',
   'x-amz-version-id': 'SUjfnDHRp9pIh.haJTMdrEVmUVunzlKU',
   'etag': '"0e99dc3dd5dd424312d45e70311c99b2"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"0e99dc3dd5dd424312d45e70311c99b2"',
 'VersionId': 'SUjfnDHRp9pIh.haJTMdrEVmUVunzlKU'}

In [31]:
#get data from bloomberg
import pandas
import urllib
import json 

def get_data(url):
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    file = response.read()
    result = json.loads(file)
    result = pandas.read_json(result)
    return result

x = get_data('https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/GOOG?period=quarter')

print(x)

ValueError: Invalid file path or buffer object type: <class 'dict'>